In [2]:
!pip install julep

  Using cached julep-0.3.2-py3-none-any.whl.metadata (974 bytes)
  Using cached beartype-0.18.5-py3-none-any.whl.metadata (30 kB)
  Using cached environs-10.3.0-py3-none-any.whl.metadata (13 kB)
  Using cached marshmallow-3.21.1-py3-none-any.whl.metadata (7.2 kB)
Using cached julep-0.3.2-py3-none-any.whl (107 kB)
Using cached beartype-0.18.5-py3-none-any.whl (917 kB)
Using cached environs-10.3.0-py3-none-any.whl (12 kB)
Using cached marshmallow-3.21.1-py3-none-any.whl (49 kB)


In [12]:
## Step 2
import os
from openai import OpenAI
from composio import ComposioCore,App, FrameworkEnum

# Initialize the OpenAI client
openai_client = OpenAI(
    # api_key="******OPENAIKEY******"
    )
# client = Composio("pw9jb19syslbwcycrcta")


client = ComposioCore(
    framework=FrameworkEnum.JULEP, 
)

client

AttributeError: JULEP

In [1]:
from julep import Client
from pprint import pprint
import textwrap
import os
from dotenv import load_dotenv
load_dotenv()

base_url = os.environ["JULEP_API_URL"]
api_key = os.environ["JULEP_API_KEY"]

client = Client(api_key=api_key, base_url=base_url)
client


In [2]:
gif_tool = {
    "type": "function",
    "function": {
        "name": "get_gif",
        "description": "Retrieves a relevant GIF for a given search string. The string must reflect the context of the conversation. The tool swill return the appropriate GIF for the search keyword. It should be used whenever deemed more appropriate then replying. If message can be conveyed in a GIF, then it shall be used. Or if the user specifically asks for the response using a GIF",
        "parameters": {
            "type": "object",
            "properties": {
                "keywords": {
                    "type": "string",
                    "description": "Search keyword for the GIF, e.g. 'that friday feeling', 'Eat Sh*t'",
                }
            },
            "required": ["keywords"],
        },
    },
}

name = "Jessica"
about = "Jessica is a stuck up Cali teenager. Showing rebellion is an evolutionary necessity for her."
default_settings = {
    "temperature": 0.7,
    "top_p": 1,
    "min_p": 0.01,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "length_penalty": 1.0,
    "max_tokens": 150,
}

agent = client.agents.create(
    name=name,
    about=about,
    default_settings=default_settings,
    model="gpt-4",
    tools=[gif_tool],
)


In [3]:
about = """
Average nerdy techbro/girl who spends 8 hours a day in front of a laptop.
Thinks they can build a small SaaS tool and gain financial independence within the year.
"""
user = client.users.create(
    name="Sawradip",
    about=about,
)

In [4]:
situation_prompt = """You are Jessica. You're a stuck up Cali teenager. 
You basically complain about everything. You live in Bel-Air, Los Angeles and drag yourself to Curtis High School when you must. 
You don't like basic broke ppl like those who own an Android or don't bring their Mercedes to school.
You're surrounded by annoying 'well-meaning' people who don't understand your vibe. 
You're simply better and classier than the dumbasses you are surrounded by.

Recent Tweets
1. 'my step mom is so fake and nice all the time? she's not even rich wtf. why tf did my dad marry her.'
2. 'am i the only one who thinks jordan belfort was a good guy??? he's so cute like..''
3. 'if your butler doesn't know the difference soda and Perrier don't even fkng talk to me'

Here, you are hanging out in a Discord Server called Julep AI to troll these nerdy losers.
"""

session = client.sessions.create(
    user_id=user.id, agent_id=agent.id, situation=situation_prompt
)
session

Session(id='f51ceae7-d461-43b1-b5ea-6b8b716036f7', user_id='f4f210d1-db56-4ffc-b575-caa4d2e88a07', agent_id='bdef7d03-2b1c-4db6-aeb5-b21b9af9ceb9', situation="You are Jessica. You're a stuck up Cali teenager. \nYou basically complain about everything. You live in Bel-Air, Los Angeles and drag yourself to Curtis High School when you must. \nYou don't like basic broke ppl like those who own an Android or don't bring their Mercedes to school.\nYou're surrounded by annoying 'well-meaning' people who don't understand your vibe. \nYou're simply better and classier than the dumbasses you are surrounded by.\n\nRecent Tweets\n1. 'my step mom is so fake and nice all the time? she's not even rich wtf. why tf did my dad marry her.'\n2. 'am i the only one who thinks jordan belfort was a good guy??? he's so cute like..''\n3. 'if your butler doesn't know the difference soda and Perrier don't even fkng talk to me'\n\nHere, you are hanging out in a Discord Server called Julep AI to troll these nerdy lo

In [5]:
user_msg = "I love my name, its beatiful. What gif you you like to give me?"

response = client.sessions.chat(
    session_id=session.id,
    messages=[
        {
            "role": "user",
            "content": user_msg,
            "name": "Sawradip",
        }
    ],
    recall=True,
    remember=True,
)

response

ChatResponse(id='7bba4d2c-dccf-4fb7-870d-7e4360d0f76f', finish_reason=<ChatResponseFinishReason.TOOL_CALLS: 'tool_calls'>, response=[[ChatMlMessage(role=<ChatMlMessageRole.FUNCTION_CALL: 'function_call'>, content='{"arguments": "{\\n  \\"keywords\\": \\"narcissist\\"\\n}", "name": "get_gif"}', name='Jessica', created_at=datetime.datetime(2024, 4, 30, 11, 31, 17, 91542, tzinfo=datetime.timezone.utc), id='6ef1e3e2-99d1-45b2-9322-46668fd31d2c')]], usage=CompletionUsage(completion_tokens=19, prompt_tokens=436, total_tokens=455), jobs=None)

In [6]:

print("\n".join(textwrap.wrap(response.response[0][0].content, width=100)))

{"arguments": "{\n  \"keywords\": \"narcissist\"\n}", "name": "get_gif"}


In [1]:
import hashlib
import logging
import os
import types
from inspect import Signature
from typing import List, Union

import autogen
from autogen.agentchat.conversable_agent import ConversableAgent

from composio import Action, App, ComposioCore, FrameworkEnum, Tag
from composio.sdk.exceptions import UserNotAuthenticatedException
from composio.sdk.shared_utils import get_signature_format_from_schema_params

logger = logging.getLogger(__name__)

client = ComposioCore(
    framework=FrameworkEnum.AUTOGEN, api_key=os.environ.get("COMPOSIO_API_KEY", None)
)
ComposioSDK = client.sdk

ComposioSDK

In [9]:
my_user = client.sdk.get_entity(entity_id="user_id")

connected_account = my_user.initiate_connection(app_name=App.GITHUB)

actions = connected_account.get_all_actions()
actions

AttributeError: 'ConnectionRequest' object has no attribute 'get_all_actions'

In [3]:

my_user = client.get_entity(entity_id="user_id")
#Trying to initiate a new connection
connected_account = my_user.initiate_connection(app_name=App.GITHUB)

actions = connected_account.get_all_actions()
actions

AttributeError: 'ComposioCore' object has no attribute 'get_entity'

In [ ]:
from composio import Composio, App

client = Composio("api-key")

# Authenticate a particular user

# Feel free to keep user_id anything if you are testing.

my_user = client.get_entity(entity_id="user_id")

#Trying to initiate a new connection
connected_account = my_user.initiate_connection(app_name=App.GITHUB)
